In [5]:
import sys
sys.path.append("../")
import torch
import torchaudio
import numpy as np
import IPython.display as ipd
from model import WaveCRN
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
output_notebook()

Loading BokehJS ...

In [6]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
# device = 'cpu'
net = WaveCRN()
net.load_state_dict(torch.load('weights.pt'))
net.to(device)
net.eval()

WaveCRN(
  (net): ConvBSRU(
    (conv): Conv1d(1, 256, kernel_size=(96,), stride=(48,), padding=(48,), bias=False)
    (deconv): ConvTranspose1d(256, 1, kernel_size=(96,), stride=(48,), padding=(48,), bias=False)
    (outfc): Linear(in_features=512, out_features=256, bias=False)
    (sru): SRU(
      (rnn_lst): ModuleList(
        (0): SRUCell(256, 256, rnn_dropout=0.1, bidirectional=True, rescale=False, layer_norm=True)
        (1): SRUCell(512, 256, rnn_dropout=0.1, bidirectional=True, rescale=False, layer_norm=True)
        (2): SRUCell(512, 256, rnn_dropout=0.1, bidirectional=True, rescale=False, layer_norm=True)
        (3): SRUCell(512, 256, rnn_dropout=0.1, bidirectional=True, rescale=False, layer_norm=True)
        (4): SRUCell(512, 256, rnn_dropout=0.1, bidirectional=True, rescale=False, layer_norm=True)
        (5): SRUCell(512, 256, rnn_dropout=0.1, bidirectional=True, rescale=False, layer_norm=True)
      )
    )
  )
)

In [8]:
noisy, sr = torchaudio.load('noisy.wav')
clean, sr = torchaudio.load('clean.wav')
pad_length = 48 - noisy.size(-1) % 48
noisy_padded = torch.nn.functional.pad(noisy, (0, pad_length))
noisy_padded.unsqueeze_(0)

tensor([[[0.0130, 0.0229, 0.0206,  ..., 0.0000, 0.0000, 0.0000]]])

In [9]:
with torch.no_grad():
    enhanced = net(noisy_padded.to(device))
enhanced = enhanced[0, :, :-pad_length]

In [11]:
p = figure(plot_width=800, plot_height=400)
x_axis = np.linspace(0, clean.size(-1) / sr, num=clean.size(-1))
p.multi_line(
    [x_axis, x_axis, x_axis],
    [enhanced[0].cpu().numpy(), clean[0].cpu().numpy(), noisy[0].cpu().numpy()],
    color=['blue', 'green', 'red'], 
    line_width=1
)

show(p)